In [44]:
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from classes import entropycalculator as ec
from tqdm import tqdm
tqdm.pandas()

wijken = gpd.read_parquet('data/wijken/wijken_stats.parquet')
buurten = gpd.read_parquet('data/buurten/buurten_stats.parquet')


In [45]:
L0_filter, L1_filter = ec.getfilter(1)

for _, wijk in wijken.iterrows():
    try:
        wijk = wijken[wijken["wijkcode"] == wijk.wijkcode]
        amenities_1 = gpd.read_parquet(f"data/gm_amenities/amenities_{wijk.gemeentenaam.values[0]}.parquet")
        amenities_1 = amenities_1[amenities_1.within(wijk.geometry.iloc[0])]
        amenities_1 = amenities_1[~amenities_1.L0_category.isin(L0_filter)]
        if L1_filter:
            for key, value in L1_filter.items():
                amenities_1 = amenities_1[~((amenities_1.L0_category == key) & (amenities_1.L1_category.isin(value)))]

        b1_outer, b1_inner = ec.return_buildings(wijk.geometry.iloc[0])
        
        fig, ax = plt.subplots(1, 1, figsize=(20, 20))
        
        b1_outer.plot(ax=ax, color="darkgray", alpha=0.3, linewidth=0.5)
        b1_inner.plot(ax=ax, color="darkgray", alpha=0.8, linewidth=0.5)
        wijk.boundary.plot(ax=ax, color="Black")
        
        amenities_1.plot(ax=ax, column="L0_category", markersize=35, legend=True)

        ax.axis("off")
        plt.savefig(f"assets/amenity_plots/wijken/{wijk.wijkcode.values[0]}.png", facecolor=fig.get_facecolor(), bbox_inches="tight", pad_inches=0, dpi=300)
        # close the plot
        plt.close()
    except Exception as e:
        print(e)
        pass
    

In [49]:
# L0_filter, L1_filter = ec.getfilter(1)

# for _, buurt in buurten.sample(5).iterrows():
#     try:
#         buurt = buurten[buurten["buurtcode"] == buurt.buurtcode]
#         amenities_1 = gpd.read_parquet(f"data/gm_amenities/amenities_{buurt.gemeentenaam.values[0]}.parquet")
#         amenities_1 = amenities_1[amenities_1.within(buurt.geometry.iloc[0])]
#         amenities_1 = amenities_1[~amenities_1.L0_category.isin(L0_filter)]
#         if L1_filter:
#             for key, value in L1_filter.items():
#                 amenities_1 = amenities_1[~((amenities_1.L0_category == key) & (amenities_1.L1_category.isin(value)))]

#         b1_outer, b1_inner = ec.return_buildings(buurt.geometry.iloc[0])
        
#         fig, ax = plt.subplots(1, 1, figsize=(20, 20))
        
#         b1_outer.plot(ax=ax, color="darkgray", alpha=0.3, linewidth=0.5)
#         b1_inner.plot(ax=ax, color="darkgray", alpha=0.8, linewidth=0.5)
#         buurt.boundary.plot(ax=ax, color="Black")
        
#         amenities_1.plot(ax=ax, column="L0_category", markersize=35, legend=True)

#         ax.axis("off")
#         plt.savefig(f"assets/amenity_plots/buurten/{buurt.buurtcode.values[0]}.png", facecolor=fig.get_facecolor(), bbox_inches="tight", pad_inches=0, dpi=300)
#         # close the plot
#         plt.close()
#     except Exception as e:
#         print(e)
#         pass